<a href="https://colab.research.google.com/github/MarkSong535/marksong535.github.io/blob/main/task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prerequisites

In [ ]:
!pip install --pre deepchem

imports

In [2]:
import deepchem as dc
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='ECFP')
train_dataset, valid_dataset, test_dataset = datasets

wcol = train_dataset.w[:,1]
train_num = np.nonzero(wcol)
x_train = train_dataset.X[train_num]
y_train = train_dataset.y[train_num][:,1]
w_train = train_dataset.w[train_num][:,1]

BATCH_SIZE = 32

wcol = valid_dataset.w[:,1]
train_num = np.nonzero(wcol)
x_val = valid_dataset.X[train_num]
y_val = valid_dataset.y[train_num][:,1]
w_val = valid_dataset.w[train_num][:,1]

wcol = test_dataset.w[:,1]
train_num = np.nonzero(wcol)
x_test = test_dataset.X[train_num]
y_test = test_dataset.y[train_num][:,1]
w_test = test_dataset.w[train_num][:,1]

print(x_val.shape)
print(x_train.shape)

train_dataset1 = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset1 = train_dataset1.shuffle(buffer_size=1024).batch(BATCH_SIZE)

val_dataset1 = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset1 = val_dataset1.shuffle(buffer_size=1024).batch(BATCH_SIZE)

test_dataset1 = tf.data.Dataset.from_tensor_slices((x_test, y_test, w_test))
test_dataset1 = test_dataset1.shuffle(buffer_size=1024).batch(BATCH_SIZE)

inputs = keras.Input(shape=(1024,), name="digits")
x1 = keras.layers.Dense(512, activation="relu")(inputs)
#x2 = keras.layers.Dense(64, activation="relu")(x1)
outputs = keras.layers.Dense(1, name="predictions")(x1)
model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()
model.compile(optimizer='sgd',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=["accuracy",tf.keras.metrics.AUC(from_logits=True)])

model.fit(train_dataset1, epochs=1000, validation_data=val_dataset1)

# Testing



(628, 1024)
(5506, 1024)
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 digits (InputLayer)         [(None, 1024)]            0         
                                                                 
 dense_4 (Dense)             (None, 512)               524800    
                                                                 
 predictions (Dense)         (None, 1)                 513       
                                                                 
Total params: 525,313
Trainable params: 525,313
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
173/173 [==============================] - 2s 7ms/step - loss: 0.3056 - accuracy: 0.9649 - auc_4: 0.3262 - val_loss: 0.1796 - val_accuracy: 0.9602 - val_auc_4: 0.6225
Epoch 2/1000
173/173 [==============================] - 1s 6ms/step - loss: 0.1826 - accuracy: 0.9649 - auc_4: 0